#Aula 2 - Volatilidade



In [1]:
#Importação e instalação de pacotes
import numpy as np
import pandas as pd
import matplotlib.pyplot as py
from scipy.stats import norm
!pip install py_vollib
import py_vollib
import warnings
warnings.filterwarnings('ignore')


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for simplejson from https://files.pythonhosted.org/packages/49/7d/94ba3f8c2b83a7536f9f62f23246e071eae15c83b5e4d9bd8a165b7f70a6/simplejson-3.19.2-cp310-cp310-macosx_11_0_arm64.whl.metadata
Using cached simplejson-3.19.2-cp310-cp310-macosx_11_0_arm64.whl (75 kB)
  Created wheel for py_vollib: filename=py_vollib-1.0.1-py3-none-any.whl size=62829 sha256=774e76704c197998506d5fa3b0fdf3de96055327d29d15c1909b58ac94a82019
  Stored in directory: /Users/ferpapi/Library/Caches/pip/wheels/84/0c/fc/b68506eda40cccaeb0013be550ba904d253ec90eae2b156052
  Created wheel for py-lets-be-rational: filename=py_lets_be_rational-1.0.1-py3-none-any.whl size=24453 sha256=9b559fdbdc2842526a68ab84e2802054c5b150ba412

##Gregas
Fonte: https://github.com/vollib/py_vollib

In [2]:
from py_vollib.black_scholes  import black_scholes as bs
from py_vollib.black_scholes.greeks.analytical import delta
from py_vollib.black_scholes.greeks.analytical import gamma
from py_vollib.black_scholes.greeks.analytical import rho
from py_vollib.black_scholes.greeks.analytical import theta
from py_vollib.black_scholes.greeks.analytical import vega

def greek_val(flag, S, K, t, r, sigma):
    price = bs(flag, S, K, t, r, sigma)
    delta_calc = delta(flag, S, K, t, r, sigma)
    gamma_calc = gamma(flag, S, K, t, r, sigma)
    rho_calc = rho(flag, S, K, t, r, sigma)
    theta_calc = theta(flag, S, K, t, r, sigma)
    vega_calc = vega(flag, S, K, t, r, sigma)
    return np.array([price, delta_calc, gamma_calc, vega_calc, theta_calc, rho_calc])

#Exemplo call
S = 50; K = 49; sigma = 0.3; T=3/12; r = 0.06

call=greek_val('c', S, K, T, r, sigma)
call

array([ 3.88268761,  0.62159964,  0.0507018 ,  0.09506588, -0.02009806,
        0.06799324])

##Volatilidade Implícita

Busca manual

In [ ]:
# Call Vanilla Europeia - B&S
def bs_call(S0, K, T, r, vol):
    d1 = (np.log(S0/K) + (r + 0.5*vol**2)*T) / (vol*np.sqrt(T))
    d2 = d1 - vol * np.sqrt(T)
    return S0 * norm.cdf(d1) - np.exp(-r * T) * K * norm.cdf(d2)

# Put Vanilla Europeia - B&S
def bs_put(S0, K, T, r, vol):
    d1 = (np.log(S0/K) + (r + 0.5*vol**2)*T) / (vol*np.sqrt(T))
    d2 = d1 - vol * np.sqrt(T)
    return np.exp(-r * T) * norm.cdf(-d2) * K - S0 * norm.cdf(-d1)

S0=11.36; K=9.21; r = 0.1165; T = 27/252
vol = 0.3 # Ajustar manualmente para casar o valor da opção
K=np.array([9.21, 9.91, 10.91, 12.91, 14.16, 15.66, 17.91]);
bs_call(S0, K, T, r, vol)
# Precisa ajusta individualente para cada valor de K

Busca por dicotomia

In [ ]:
def busca_vol_imp(preço_alvo, S, K, T, r):
    max_iter = 200
    precisao = 1.0e-5
    vol_imp = np.zeros(len(K))
    for j in range(0,len(K)):
        a=0 # inicio do intervalo
        b=5 # fim do intervalo
        vol_imp[j] = (a+b)/2
        for i in range(0, max_iter):
            price = bs_call(S, K[j], T, r, vol_imp[j])
            diff = preço_alvo[j] - price
            if (abs(diff) < precisao):
                break
            if diff < 0:
                a,b = a,vol_imp[j]
            elif diff > 0:
                a,b = vol_imp[j],b
            vol_imp[j] =(a+b)/2
    return vol_imp

S=11.36; K=np.array([9.21, 9.91, 10.91, 12.91, 14.16, 15.66, 17.91]); r = 0.1165; vol = 0.3; T = 27/252
V_market = np.array([2.46, 1.85, 1.09, 0.38, 0.19, 0.09, 0.03])
imp_vol = busca_vol_imp(V_market, S, K, T, r)
py.plot(V_market, imp_vol)
py.show
imp_vol

Busca com pacote py_vollib

In [ ]:
from py_vollib.black_scholes.implied_volatility import implied_volatility as iv

S=11.36; K=np.array([9.21, 9.91, 10.91, 12.91, 14.16, 15.66, 17.91]); r = 0.1165; vol = 0.3; T = 27/252
V_market = np.array([2.46, 1.85, 1.09, 0.38, 0.19, 0.09, 0.03])
vol_imp = np.zeros(len(K))

for j in range(0, len(K)):
    vol_imp[j] = iv(V_market[j], S, K[j], T, r, 'c')

py.plot(V_market, vol_imp)
py.title("Vol. Implícita")
py.xlabel("Prêmio")
py.ylabel("Vol");
py.show
imp_vol